In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime

In [24]:
start_date = datetime.date(2014, 1, 1)
end_date = datetime.date(2024, 9, 30)
date_range = pd.date_range(start=start_date, end=end_date)
df = pd.DataFrame({"date": date_range,
                   "pm2.5": [None] * len(date_range),
                   "pm10": [None] * len(date_range),
                   "o3": [None] * len(date_range),
                   "no2": [None] * len(date_range),
                   "so2": [None] * len(date_range),
                   "co": [None] * len(date_range)})

In [33]:
path = "D:\chromedriver-win64\chromedriver.exe"
s = Service(path)
driver = webdriver.Chrome(service=s)
driver.get("https://aqicn.org/historical/#city:vietnam/da-nang")
time.sleep(3)
# find position of (pm2.5, pm10, o3, no2, so2, co)
indexes = driver.find_elements(By.XPATH, "//*[@id='historical-aqidata-header']/div[2]/div/center/span[1]/button")

try:
    for i in indexes:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", i)
        i.click()
        param = i.text.lower()
        actions = ActionChains(driver)
        search = driver.find_element(By.XPATH, "//*[@id='historical-aqidata-header']/div[2]/div/center[2]/div[3]")
        search = search.find_elements(By.TAG_NAME, "svg")
        for element in search:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'})", element)
            year = element.find_elements(By.CSS_SELECTOR, "rect.rect-day-median")
            time.sleep(2)
            for day in year:
                actions.move_to_element(day).perform()
                val = driver.find_element(By.CLASS_NAME, "calendar-tooltip").text
                if val == "":
                    continue
                val = val.split()
                if len(val) == 1:
                    continue
                dates = pd.Timestamp(val[0])
                print(param, val)
                df.loc[df["date"] == dates, param] = int(val[2][7:])
                if dates == pd.Timestamp("2024-09-30"):
                    break
except Exception as e:
    print(e)
time.sleep(3)
driver.quit()

pm2.5 ['2024-01-01', 'average', '(mean):181']
pm2.5 ['2024-01-02', 'average', '(mean):144']
pm2.5 ['2024-01-03', 'average', '(mean):114']
pm2.5 ['2024-01-04', 'average', '(mean):91']
pm2.5 ['2024-01-05', 'average', '(mean):40']
pm2.5 ['2024-01-06', 'average', '(mean):38']
pm2.5 ['2024-01-07', 'average', '(mean):32']
pm2.5 ['2024-01-08', 'average', '(mean):39']
pm2.5 ['2024-01-09', 'average', '(mean):30']
pm2.5 ['2024-01-10', 'average', '(mean):46']
pm2.5 ['2024-01-11', 'average', '(mean):52']
pm2.5 ['2024-01-12', 'average', '(mean):33']
pm2.5 ['2024-01-13', 'average', '(mean):31']
pm2.5 ['2024-01-14', 'average', '(mean):46']
pm2.5 ['2024-01-15', 'average', '(mean):38']
pm2.5 ['2024-01-16', 'average', '(mean):25']
pm2.5 ['2024-01-17', 'average', '(mean):24']
pm2.5 ['2024-01-18', 'average', '(mean):34']
pm2.5 ['2024-01-26', 'average', '(mean):7']
pm2.5 ['2024-01-27', 'average', '(mean):9']
pm2.5 ['2024-01-28', 'average', '(mean):13']
pm2.5 ['2024-01-29', 'average', '(mean):9']
pm2.5 ['20

In [34]:
df.head(15)

,date,pm2.5,pm10,o3,no2,so2,co
0,2014-01-01,None,None,None,None,None,None
1,2014-01-02,None,44,73,28,92,37
2,2014-01-03,125,40,67,17,93,20
3,2014-01-04,117,39,37,23,92,12
4,2014-01-05,116,None,None,None,None,None
5,2014-01-06,None,36,100,12,93,16
6,2014-01-07,105,32,69,15,93,21
7,2014-01-08,99,26,46,12,93,12
8,2014-01-09,84,25,84,15,93,17
9,2014-01-10,83,19,72,18,93,21


In [35]:
df.to_csv("air_pollution_data.csv", index=False)